# Part 3: Fibonacci Example
# Chapter 3: Witness
Now, we will generate multiple witnesses to test the soundness of our circuit constraints. Note that we only intend to accept the following set of values for signals "a", "b", and "c". "Soundness" in this context refers to faulty witness successfully verified against the constraints (false positives), so any set of witness assignments that is different from the table below but still passes the constraints incurs a "soundness" error.
| Step Type | Step Instance Index || Signals ||| Setups ||
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
||| a | b | c | constraint 1 | constraint 2 | constraint 3 |
| fibo step | 0 | 1 | 1 | 2 | a + b == c | b == a.next | c == b.next |
| fibo step | 1 | 1 | 2 | 3 | a + b == c | b == a.next | c == b.next |
| fibo step | 2 | 2 | 3 | 5 | a + b == c | b == a.next | c == b.next |
| fibo step | 3 | 3 | 5 | 8 | a + b == c | b == a.next | c == b.next |
| ... | ... || ... ||| ... ||

## Setup
We setup the same circuit and witness in Part 1 which were successfully verified:

In [11]:
from dsl import Circuit, StepType
from cb import eq
from util import F

class FiboStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

class Fibonacci(Circuit):
    def setup(self):
        self.a = self.forward("a")
        self.b = self.forward("b")
        
        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        self.pragma_first_step(self.fibo_step)
        self.pragma_num_steps(4)
        
    def trace(self, args):
        self.add(self.fibo_step, (1, 1))
        a = 1
        b = 2
        for i in range(1, 4):
            self.add(self.fibo_step, (a, b))
            prev_a = a
            a = b
            b += prev_a

fibo = Fibonacci()
fibo_witness = fibo.gen_witness(None)
fibo.halo2_mock_prover(fibo_witness)

there's last step
4
1
237409307236249817799321350805519993354
Err(
    [
        ConstraintCaseDebug {
            constraint: Constraint {
                gate: Gate {
                    index: 0,
                    name: "main",
                },
                index: 3,
                name: "q_first => Product(Fixed { query_index: 2, column_index: 1, rotation: Rotation(0) }, Sum(Constant(0x0000000000000000000000000000000000000000000000000000000000000001), Negated(Advice { query_index: 0, column_index: 3, rotation: Rotation(0) })))",
            },
            location: InRegion {
                region: Region 0 ('circuit'),
                offset: 0,
            },
            cell_values: [
                (
                    DebugVirtualCell {
                        name: "",
                        column: DebugColumn {
                            column_type: Advice,
                            index: 3,
                            annotation: "'step selector for fibo_s

Now we swap the first step instance from `(1, 1, 2)` to `(0, 2, 2)`. We use the `evil_witness` function to swap step index 0 assignment index 0 to `F(0)` and step index 0 assignment index 0 to `F(2)`.

In [12]:
evil_witness = fibo_witness.evil_witness_test(step_instance_indices=[0, 0], assignment_indices=[0, 1], rhs=[F(0), F(2)])

Print the `evil_witness` to confirm that the swap was successful:

In [13]:
print(evil_witness)

TraceWitness(
	step_instances={
		StepInstance(
			step_type_uuid=237407242550334696074761953987734931978,
			assignments={
				a = 0,
				b = 2,
				c = 2
			},
		),
		StepInstance(
			step_type_uuid=237407242550334696074761953987734931978,
			assignments={
				a = 1,
				b = 2,
				c = 3
			},
		),
		StepInstance(
			step_type_uuid=237407242550334696074761953987734931978,
			assignments={
				a = 2,
				b = 3,
				c = 5
			},
		),
		StepInstance(
			step_type_uuid=237407242550334696074761953987734931978,
			assignments={
				a = 3,
				b = 5,
				c = 8
			},
		)
	},
)


Now, generate and verify the proof with `evil_witness`:

In [14]:
fibo.halo2_mock_prover(evil_witness)

Err(
    [
        ConstraintCaseDebug {
            constraint: Constraint {
                gate: Gate {
                    index: 0,
                    name: "main",
                },
                index: 3,
                name: "q_first => Product(Fixed { query_index: 2, column_index: 1, rotation: Rotation(0) }, Sum(Constant(0x0000000000000000000000000000000000000000000000000000000000000001), Negated(Advice { query_index: 0, column_index: 3, rotation: Rotation(0) })))",
            },
            location: InRegion {
                region: Region 0 ('circuit'),
                offset: 0,
            },
            cell_values: [
                (
                    DebugVirtualCell {
                        name: "",
                        column: DebugColumn {
                            column_type: Advice,
                            index: 3,
                            annotation: "'step selector for fibo_step'",
                        },
                        rota

Surprisingly, `evil_witness` generated a proof that passed verification. This constitutes a soundness error, because the first step instance isn't `(1, 1, 2)` as we initially specified, so why can the witness still pass the constraints?

The answer is simple, because in the first step instance, we never constrained the values of "a" and "b" to 1 and 1 in `setup` of `FiboStep`. We also didn't constrain the first step instance to be `FiboStep`.

You might be wondering: in `trace`, didn't we set "a" and "b" to `(1, 1)` and added `FiboStep` as the first step instance? In fact, `trace` and `wg` are really helper functions for the prover to easily generate a witness, whose data can be tampered with as shown in `evil_witness_test`. The only conditions enforced are defined in circuit and step type `setup`. Therefore, to fix the soundness error, we need to add more constraints, in Chapter 4.